In [78]:
%load_ext autoreload
%autoreload 2
import RNA
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
from random import choices, seed

from utils import *
from penalties import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
seq_db_pairs = parse_dp_file('RNAstrand/RNAstrand_noMS_noPK.dp')

In [45]:
# Get 10 random structures
seed(1337)
sample = choices(list(seq_db_pairs.keys()), k=10)
for s in sample:
    print(s, len(seq_db_pairs[s]['seq']))

SPR_00327 78
SPR_00167 72
RFA_00678 44
SPR_00264 85
PDB_00150 25
SRP_00080 290
RFA_00709 45
SRP_00013 305
SRP_00252 314
RFA_00431 40


In [47]:
# See where the best structures happen
percents = np.arange(0.00, 1.05, 0.05)
md = RNA.md()
md.temperature = 37
samples = {}
for k in sample:
    seq = seq_db_pairs[k]['seq']
    ref = seq_db_pairs[k]['db']
    print("Working on", k, "len =", len(seq))

    samples[k] = {}

    for p in percents:
        print("Working on percentage = {:.3f}".format(p))
        samples[k][p] = {}
        last_structure = ''
        for length in range(10, len(seq)):
            subseq = seq[:length+1]

            samples[k][p][length+1] = sequence_dependent_penalty(subseq, p, last_structure, md)[0]
            last_structure = samples[k][p][length+1]




Working on SPR_00327 len = 78
Working on percentage = 0.000
Working on percentage = 0.050
Working on percentage = 0.100
Working on percentage = 0.150
Working on percentage = 0.200
Working on percentage = 0.250
Working on percentage = 0.300
Working on percentage = 0.350
Working on percentage = 0.400
Working on percentage = 0.450
Working on percentage = 0.500
Working on percentage = 0.550
Working on percentage = 0.600
Working on percentage = 0.650
Working on percentage = 0.700
Working on percentage = 0.750
Working on percentage = 0.800
Working on percentage = 0.850
Working on percentage = 0.900
Working on percentage = 0.950
Working on percentage = 1.000
Working on SPR_00167 len = 72
Working on percentage = 0.000
Working on percentage = 0.050
Working on percentage = 0.100
Working on percentage = 0.150
Working on percentage = 0.200
Working on percentage = 0.250
Working on percentage = 0.300
Working on percentage = 0.350
Working on percentage = 0.400
Working on percentage = 0.450
Working on

In [75]:
x = []
y = []
ymin = []
ymax = []
c = []
for k in samples.keys():
    length = len(seq_db_pairs[k]['seq'])
    best_score = -1
    best_percent = 0
    mccs = []
    #print(k)
    for p in percents:
        mcc = calc_MCC(samples[k][p][length], seq_db_pairs[k]['db'])
        mccs.append(mcc)
        #print(f"{p:.3f}", f"{mcc:.3f}")
        if mcc >= best_score:
            best_score = mcc
            best_percent = p

    # For most of these, the best structure happened over a range
    mccs = np.array(mccs)
    mask = mccs == best_score
    yrange = percents[mask]
    ymin.append(min(yrange))
    ymax.append(max(yrange))

    x.append(best_score)
    y.append(best_percent)
    c.append(length)

In [83]:
cmap = mpl.colormaps['viridis']
cols = [cmap(l/max(c)) for l in c]
fig, ax = plt.subplots()
#s = ax.scatter(x, y, c=c)
bars = ax.vlines(x, ymin, ymax, colors=cols)

ax.set_xlabel('MCC')
ax.set_ylabel('Penalty multiplier')
cbar = ax.figure.colorbar(bars, ax=ax)
cbar.ax.set_ylabel('Sequence length', rotation=-90, va="bottom")

plt.show()

TypeError: Colormap.set_extremes() takes 1 positional argument but 2 were given